### 容器 & 迭代器
- 容器:对象的集合--列表、元组、字典、集合
    - 所有的容器都是可迭代的
- 迭代器(iterator):提供了一个next方法, 返回容器的下一个对象 or StopIteration错误
    - 可迭代对象通过iter(x)返回一个迭代器--再通过next()实现遍历--for in 将这个过程隐式化
    - 判断是否可迭代:
        - isinstance(obj, Iterable)
    - [i for i in range(num)]
### 迭代器 & 生成器
- 生成器: 懒人版的迭代器--不需要事先生成,在调用next()函数的时候才会生成下一个变量
    - (i for i in range(num))
    - yield: 函数到这一行暂停,然后跳出到next(),每次next()被调用暂停复活
    - 迭代器是一个有限集合,生成器则可以成为一个无限集--next触发
    - 合理使用生成器,可以降低内存占用、优化程序结构、提高程序速度
    - 生成器在 Python 2 的版本上，是协程的一种重要实现方式；而 Python 3.5 引入 async await 语法糖后，生成器实现协程的方式就已经落后了
    - 对于一个有限元素的生成器，如果迭代完成后，继续调用 next()会 raise StopIteration。只有复位生成器才能重新进行遍历。

In [13]:
def is_iterable(param):
    try: 
        iter(param) 
        return True
    except TypeError:
        return False

params = [
    1234,
    '1234',
    [1, 2, 3, 4],
    set([1, 2, 3, 4]),
    {1:1, 2:2, 3:3, 4:4},
    (1, 2, 3, 4)
]
    
for param in params:
    print('{} is iterable? {}'.format(param, is_iterable(param)))


1234 is iterable? False
1234 is iterable? True
[1, 2, 3, 4] is iterable? True
{1, 2, 3, 4} is iterable? True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True
(1, 2, 3, 4) is iterable? True


In [6]:
import os
import psutil

def show_memory_info(message):
    pid = os.getpid()
    p = psutil.Process(pid)
    info = p.memory_full_info()
    memory = info.uss / 1024 / 1024
    print('{} memory used: {} MB'.format(message, memory))

def test_iterator():
    show_memory_info('initiating iterator')
    list_1 = [i for i in range(100000)]
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')

def test_generator():
    show_memory_info('initiating generator')
    list_1 = (i for i in range(100000))
    show_memory_info('after generator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')

test_iterator()
test_generator()

initiating iterator memory used: 15.4375 MB
after iterator initiated memory used: 18.4375 MB
4999950000
after sum called memory used: 18.890625 MB
initiating generator memory used: 16.890625 MB
after generator initiated memory used: 16.890625 MB
4999950000
after sum called memory used: 16.890625 MB


In [80]:
def generator(k):
    i = 1
    while True:
        yield i ** k
        i += 1

gen_1 = generator(1)
gen_3 = generator(3)
print(gen_1)
print(gen_3)

def get_sum(n):
    sum_1, sum_3 = 0, 0
    for i in range(n):
        next_1 = next(gen_1)
        next_3 = next(gen_3)
        print('next_1={}, next_3={}'.format(next_1, next_3))
        sum_1 += next_1
        sum_3 += next_3
    print(sum_1 * sum_1, sum_3)

get_sum(8)

<generator object generator at 0x1048a4700>
<generator object generator at 0x103993f40>
next_1=1, next_3=1
next_1=2, next_3=8
next_1=3, next_3=27
next_1=4, next_3=64
next_1=5, next_3=125
next_1=6, next_3=216
next_1=7, next_3=343
next_1=8, next_3=512
1296 1296


#### 给定一个 list 和一个指定数字，求这个数字在 list 中的位置。

In [145]:
def index_normal(L, target):
    result = []
    for i, num in enumerate(L):
        if num == target:
            result.append(i)
    return result

print(index_normal([1,6,2,4,5,2,8,6,2,3], 2))

def index_generator(L, target):
    for i, num in enumerate(L):
        if num == target:
            yield i
# list会自动迭代生成器
print(list(index_generator([1,6,2,4,5,2,8,6,2,3], 2)))

[2, 5, 8]
[2, 5, 8]


#### 给定两个序列，判定第一个是不是第二个的子序列。
序列就是列表，子序列则指的是，一个列表的元素在第二个列表中都按顺序出现，但是并不必挨在一起。

In [235]:
def is_subsequence(a, b):
    b = iter(b)
    return all(i in b for i in a)

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

True
False


In [231]:
def is_subsequence(a, b):
    b = iter(b)
    print(b)

    gen = (i for i in a)
    print(gen)
    for i in gen:
        print(i)
    # (i in b) ==> while True:
    #                 val = next(b)
    #                 if val == i:
    #                    yield True
    gen = ((i in b) for i in a)
    print(gen)
    for i in gen:
        print(i)

    return all(((i in b) for i in a))

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

<generator object is_subsequence.<locals>.<genexpr> at 0x103acca00>
1
3
5
<generator object is_subsequence.<locals>.<genexpr> at 0x10495cad0>
True
True
True
False
<generator object is_subsequence.<locals>.<genexpr> at 0x103acca00>
1
4
3
<generator object is_subsequence.<locals>.<genexpr> at 0x10495cad0>
True
True
False
False


In [232]:
b = (i for i in range(5))

print(2 in b)
print(4 in b)
print(3 in b)


True
True
False
